In [1]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0)
ERROR: No matching distribution found for tensorflow==2.4.1


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
cap = cv2.VideoCapture(0)
results=[]
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
len(results.left_hand_landmarks.landmark)

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
from tqdm import tqdm

TRAIN_DIR = 'D:/Agrima/animals_video'
SAVE_DIR = 'D:/Agrima/animals_npy'
IM_DIR = 'D:/Agrima/animals_IM'
TRAIN_DIR1 = 'D:\Agrima\animals_video'
SAVE_DIR1 = 'D:\Agrima\animals_npy'
CATEGORIES = []
train_dir = os.listdir(TRAIN_DIR)
IMG_SIZE = 224
for folder in train_dir:
    CATEGORIES.append(folder)
cat_len=len(CATEGORIES)
    
# # for category in CATEGORIES:
# #     for img in tqdm(os.listdir(TRAIN_DIR)):
# #         try: 
# #             os.makedirs(os.path.join(SAVE_DIR1, category, img))
# #         except:
# #             pass
        
# '''Creating the training data'''
# # def create_train_data():
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     for category in CATEGORIES:
#         path=TRAIN_DIR+"/"+category
#         path2=SAVE_DIR+"/"+category
#         path3=IM_DIR+"/"+category
#         if not os.path.exists(path2): 
#             os.makedirs(path2) 
#         if not os.path.exists(path3): 
#             os.makedirs(path3) 

#         for img in tqdm(os.listdir(path)):
#             path12=path2+"/"+str(img)
#             path13=path3+"/"+str(img)
#             if not os.path.exists(path12): 
#                 os.makedirs(path12)
#             if not os.path.exists(path13): 
#                 os.makedirs(path13)
#             img_array = cv2.VideoCapture((path+"/"+img))
#              # Read feed
#             i = 0
#             while(True):
#                 ret,frame = img_array.read()
#                 if ret:
#                     frame = cv2.resize(frame,(IMG_SIZE,IMG_SIZE),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
#                 # Make detections
#                     image, results = mediapipe_detection(frame, holistic)
#                     draw_styled_landmarks(image, results)
#                     im_path = path13 +"/"+ str(i) + ".jpg"
#                     cv2.imwrite(im_path, image)
#                 # Draw landmarks
#                 #draw_styled_landmarks(image, results)
#                     keypoints = extract_keypoints(results)
#                     npy_path = path12 +"/"+ str(i) 
#                     np.save(npy_path, keypoints)
#                     i = i + 1
#                 else:
#                     break


# img_array.release()
# cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(CATEGORIES)}

sequences, labels, res = [], [], []
for category in CATEGORIES:
    path=TRAIN_DIR+"/"+category
    path2=SAVE_DIR+"/"+category
    for img in tqdm(os.listdir(path)):
        path12=path2+"/"+str(img)
        window = []
        for i in range(28):
            res = np.load(path12+"/"+ str(i)+".npy")
            
           #print(path12+"/"+ str(i)+".npy")
            window.append(res)
        sequences.append(window)
        labels.append(label_map[category])
        

100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:01<00:00, 78.12it/s]


In [9]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [11]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(28,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(CATEGORIES), activation='softmax'))

In [26]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28,1662,1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(cat_len,  activation=tf.nn.softmax)
])
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras import callbacks
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint("D:/Agrima/animals_video/mp_4layer.h5", monitor="val_loss", verbose=1,
                              save_best_only=True, mode='auto', period=1)
reduce = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, 
                           min_lr=0.0001, verbose = 1)
earlystopping = callbacks.EarlyStopping(monitor="val_loss", patience = 5, restore_best_weights = True)


In [ ]:
history = model.fit(X_train, y_train, batch_size = 16, epochs = 40, verbose = 1, validation_data = (X_test, y_test),
                   callbacks= [earlystopping,checkpoint])


In [14]:
model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test), callbacks=[checkpoint,reduce])

NameError: name 'checkpoint' is not defined

In [8]:
from keras.models import load_model
model = load_model('D:/Agrima/animals_video/mp_4layer.h5')

In [17]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)


array([[[391,   1],
        [  0,   9]],

       [[370,   1],
        [  0,  30]],

       [[370,   0],
        [  1,  30]],

       [[356,   7],
        [  2,  36]],

       [[362,   1],
        [  0,  38]],

       [[364,   7],
        [  0,  30]],

       [[352,   1],
        [  5,  43]],

       [[361,   1],
        [  0,  39]],

       [[370,   2],
        [  1,  28]],

       [[363,   0],
        [  1,  37]],

       [[361,   0],
        [  5,  35]],

       [[369,   1],
        [  7,  24]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

0.85785536159601

In [70]:
color = (245,117,16)
def prob_viz(res, CATEGORIES, input_frame, colors):
    output_frame = input_frame.copy()
#     np.sort(res)[::-1]
#     print((res))
    for num, prob in enumerate(res):
        #print(num,prob)
        cv2.rectangle(output_frame, (0,60+num*30), (int(prob*100), 90+num*30), color, -1)
        cv2.putText(output_frame, CATEGORIES[num], (0, 80+num*30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [37]:
img_array = cv2.imread(('D:/Agrima/animals_SpatialData/test/cow/1.jpg'))
img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, CATEGORIES, img_array, colors))

<Figure size 1296x1296 with 0 Axes>

<Figure size 1296x1296 with 0 Axes>

In [21]:
# # 1. New detection variables
# sequence = []
# sentence = []
# threshold = 0.8

# import time


# # SET THE COUNTDOWN TIMER
# # for simplicity we set it to 3
# # We can also take this as input
# TIMER = int(3)
# cap = cv2.VideoCapture(0)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # Read feed
#         ret, frame = cap.read()
#         cv2.imshow('a',frame)
#         k = cv2.waitKey(125)
#         if k == ord('w'):
#             prev = time.time()
#             while TIMER >= 0:
#                 ret, frame = cap.read()
#                 font = cv2.FONT_HERSHEY_SIMPLEX
#                 cv2.putText(frame, str(TIMER),
#                             (200, 250), font,
#                             7, (0, 255, 255),
#                             4, cv2.LINE_AA)
#                 cv2.imshow('a', frame)
#                 cv2.waitKey(125)
#                 cur = time.time()
#                 if cur-prev >= 1:
#                     prev = cur
#                     TIMER = TIMER-1

#             else:
#                 ret, frame = cap.read()
#                 cv2.imshow('a', frame)
#                 image, results = mediapipe_detection(frame, holistic)
#                 draw_styled_landmarks(image, results)
#                 keypoints = extract_keypoints(results)
#                 sequence.append(keypoints)
#                 print(len(sequence))
#                 sequence = sequence[-28:]
#                 print(len(sequence))
#                 print("3333nging")
        
#                 if len(sequence) == 28:
#                     print("4444nging")
#                     res = model.predict(np.expand_dims(sequence, axis=0))[0]
#                     print(CATEGORIES[np.argmax(res)])
#                     if res[np.argmax(res)] > threshold: 
#                         if len(sentence) > 0: 
#                             if CATEGORIES[np.argmax(res)] != sentence[-1]:
#                                 sentence.append(CATEGORIES[np.argmax(res)])
#                         else:
#                             sentence.append(CATEGORIES[np.argmax(res)])
#                     if len(sentence) > 5: 
#                         sentence = sentence[-5:]
#                     image = prob_viz(res, CATEGORIES, image, colors)
#                 cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
#                 cv2.putText(image, ' '.join(sentence), (3,30), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#                 cv2.imshow('b', image)

#         # Break gracefully
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

1
1
3333nging


In [79]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-28:]
        
        if len(sequence) == 28:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(CATEGORIES[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                
                if len(sentence) > 0: 
                    if CATEGORIES[np.argmax(res)] != sentence[-1]:
                        sentence.append(CATEGORIES[np.argmax(res)])
                else:
                    sentence.append(CATEGORIES[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            #Viz probabilities
            image = prob_viz(res, CATEGORIES, image, color)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image,CATEGORIES[np.argmax(res)] , (3,30), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [78]:
print((res))

[8.5686316e-04 2.5739256e-02 2.5478982e-08 3.3693625e-05 1.0658009e-03
 2.7856296e-01 1.1805955e-02 1.3177223e-05 2.5914696e-01 9.1112591e-04
 1.2216684e-01 2.9969734e-01]
